results of each tracking are in video files

In [4]:
import kagglehub
path = kagglehub.model_download("tensorflow/ssd-mobilenet-v2/tensorFlow2/fpnlite-320x320")
print("Path to model files:", path)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

  0%|          | 0.00/6.74k [00:00<?, ?B/s]



100%|██████████| 6.74k/6.74k [00:00<00:00, 764kB/s]








100%|██████████| 8.83M/8.83M [00:06<00:00, 1.40MB/s]


100%|██████████| 10.3M/10.3M [00:06<00:00, 1.55MB/s]

Path to model files: /Users/home/.cache/kagglehub/models/tensorflow/ssd-mobilenet-v2/tensorFlow2/fpnlite-320x320/1


In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import time
import os

def list_connected_devices(max_devices=10):
    available_devices = []
    for device_index in range(max_devices):
        cap = cv2.VideoCapture(device_index)
        if cap.isOpened():
            available_devices.append(device_index)
            cap.release()
    return available_devices
#print("Connected video devices:", list_connected_devices())

# Load pre-trained MobileNet-SSD model
model = tf.saved_model.load("saved_model")

# Function to perform human detection and distance estimation
def detect_and_estimate_distance(frame, threshold_area):
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis,...]
    
    detections = model(input_tensor)

    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)
    scores = detections['detection_scores'][0].numpy()

    frame_height, frame_width, _ = frame.shape

    for i in range(len(scores)):
        if scores[i] > 0.5 and classes[i] == 1:  # Class 1 corresponds to 'person'
            ymin, xmin, ymax, xmax = boxes[i]
            (left, right, top, bottom) = (xmin * frame_width, xmax * frame_width, ymin * frame_height, ymax * frame_height)
            area = (right - left) * (bottom - top)
            
            # Draw bounding box
            cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 2)
            
            # Check if area exceeds threshold
            if area > threshold_area:
                return True, frame
    
    return False, frame

# Access webcam and process frames
cap = cv2.VideoCapture(0)
threshold_area = 50000  # Define an appropriate threshold area

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    is_close, processed_frame = detect_and_estimate_distance(frame, threshold_area)

    if is_close:
        # Run drone.py
        cv2.imshow("Frame", processed_frame)
        os.system('python drone.py')
        # Pause execution for 1 minute
        time.sleep(60)
        break
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2024-07-11 18:41:40.197979: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
sh: python: command not found
